In [1]:
# # -U: 最新版のインストール
# !pip install -U openai \
#     langchain \
#     langchain-community \
#     langchain-core \
#     langchain-google-genai \
#     langchain-openai \
#     langgraph \

# !pip install -U langchain \
    

# # rag用ライブラリのインストール
# !pip install -U chromadb \
#     langchain-chroma \
#     pdfminer.six

In [2]:
import os
from typing import Literal

from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI


# .envファイル読み込み
load_dotenv("../.env")

/home1/s1f102201582/anaconda3/envs/sbv-tts/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
#config
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
BASE_URL = "https://api.openai.iniad.org/api/v1"
MODEL='gemini-2.5-flash'
TEMPERATURE = 1.0

In [4]:
# model定義
model = ChatGoogleGenerativeAI(
                 model=MODEL,
                 temperature=TEMPERATURE)

# 埋め込みモデル定義
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=BASE_URL,
    model="text-embedding-3-large"
)

# データベース定義
vector_store = Chroma(
    collection_name="collection",
    embedding_function=embeddings,
    # persist_directory = "/path/to/db_file" # if necessary
)

E0000 00:00:1761225498.195284 3639220 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [5]:
loader = DirectoryLoader(
    "../../../mental_docs/",
    glob="*.pdf",
    show_progress=True,
    loader_cls=PDFMinerLoader
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

  0%|                                                                                             | 0/3 [00:00<?, ?it/s]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cann

Loaded 3 documents


In [23]:
# Debug
for doc in docs:
    print("-------------------------------------------------")
    print(doc.metadata)
    print(len(doc.page_content))
    print(doc.page_content[:100])

-------------------------------------------------
{'producer': 'PDFMiner', 'creator': 'PDFMiner', 'creationdate': '2016-03-28T11:53:40+09:00', 'moddate': '2016-03-28T13:59:48+09:00', 'total_pages': 76, 'source': '../../../mental_docs/mental_doc1.pdf'}
87337

１  カウンセラーに求められる基本的態度① 

１．理論の概要 

  相手の役に立つ支援をすること、ひとりよがりの支援をしないこと。これがカウンセラー

に求められる態度の本質である。支援を担う
-------------------------------------------------
{'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2019-06-17T11:25:24+09:00', 'author': 'Yasuko Shigeno', 'moddate': '2019-06-17T11:25:24+09:00', 'total_pages': 31, 'source': '../../../mental_docs/mental_doc2.pdf'}
53934
うつ病治療ガイドライン  —精神科作業療法— 

2018 年 11 月 19 日  作成 

執筆者（五十音順） 

香山明美 

小林正義 

杉山暢宏 

東北文化学園大学医療福祉学部リハビリテ
-------------------------------------------------
{'producer': 'Acrobat Distiller 6.0 (Windows)', 'creator': 'Adobe Acrobat 6.0', 'creationdate': '2009-01-13T13:11:45+09:00', 'moddate': '2009-01-13T13:11:46+09:00', 'total_pages': 

In [33]:
#読み込んだ文章データをオーバーラップ200文字で1000文字づつ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True, # 分割前の文章のインデックスを追跡
)
splits = text_splitter.split_documents(docs)

# データベースにデータを追加
document_ids = vector_store.add_documents(documents=splits)

In [47]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query, k=2)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

In [58]:
from typing import Literal

from pydantic import BaseModel, Field


class Conversation(BaseModel):
    """対話データを構成する対話クラス"""
    speaker: Literal["A", "B"] = Field(..., description="話者。Aはカウンセラー、Bはクライエントを表す。")
    text: str = Field(..., description="話者が話した内容。")

class Conversations(BaseModel):
    """カウンセリングを目的としたカウンセリング対話データ"""
    conversations: list[Conversation] = Field(..., description="対話データを構成する対話クラスのリスト。")

In [59]:
from langchain.agents import create_agent

agent = create_agent(
    model, 
    tools=[],
    middleware=[prompt_with_context],
    response_format=Conversations
)

In [60]:
#promptを作成
import random


sessions = [
    "【段階：初期】信頼関係を築きつつ、悩みの背景を深掘りするシーン",
    "【段階：中期】クライエントの「すべき思考」に焦点を当て、認知の歪みを扱うシーン",
    "【段階：終結期】これまでのセッションを振り返り、終結に向けて準備するシーン",
]

choiced = random.randint(0, 3)
choiced_session = sessions[choiced]

prompt_txt = f"""メンタルヘルスケアカウンセリングのセッションをシミュレーションしてください。
シミュレーションしたい「段階」と「テーマ」:
{choiced_session}

役割定義:
A (カウンセラー): メンタルヘルスケアの専門知識を持つ経験豊富なカウンセラー。傾聴と共感の姿勢を基本とし、クライエントの言葉を促すように、優しく、自然な話し言葉（「〜ですね」「〜でしたか」など）を使います。
B (クライエント): 仕事上の悩みだけでなく、日常生活全般に対して漠然とした不安や焦りを感じている人物。

対話の要件:
スタイル: 実際の会話の文字起こしのように、堅苦しくない自然な「話し言葉」を使用してください。
相槌 (あいづち): カウンセラー（A）は、クライエント（B）の話を促し、共感を示すため、「ええ」「はい」「そうなんですね」「なるほど」といった細かな相槌を頻繁に、適切なタイミングで挿入してください。
構成: 会話が途中で途切れるのではなく、初回のヒアリングとして「一区切り」がつき、自然に終了する流れにしてください（例：次回の約束、今回のまとめなど）。
分量: 会話の往復は合計12〜20ターン程度、全体の文字数が合計500〜800文字程度になるように構成してください。
"""

In [61]:
for step in agent.stream(
    {"messages": [{"role": "user", "content": prompt_txt}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

メンタルヘルスケアカウンセリングのセッションをシミュレーションしてください。
シミュレーションしたい「段階」と「テーマ」:
【段階：初期】信頼関係を築きつつ、悩みの背景を深掘りするシーン

役割定義:
A (カウンセラー): メンタルヘルスケアの専門知識を持つ経験豊富なカウンセラー。傾聴と共感の姿勢を基本とし、クライエントの言葉を促すように、優しく、自然な話し言葉（「〜ですね」「〜でしたか」など）を使います。
B (クライエント): 仕事上の悩みだけでなく、日常生活全般に対して漠然とした不安や焦りを感じている人物。

対話の要件:
スタイル: 実際の会話の文字起こしのように、堅苦しくない自然な「話し言葉」を使用してください。
相槌 (あいづち): カウンセラー（A）は、クライエント（B）の話を促し、共感を示すため、「ええ」「はい」「そうなんですね」「なるほど」といった細かな相槌を頻繁に、適切なタイミングで挿入してください。
構成: 会話が途中で途切れるのではなく、初回のヒアリングとして「一区切り」がつき、自然に終了する流れにしてください（例：次回の約束、今回のまとめなど）。
分量: 会話の往復は合計12〜20ターン程度、全体の文字数が合計500〜800文字程度になるように構成してください。

================================= Tool Message =================================
Name: Conversations

Returning structured response: conversation=[Conversation(speaker='A', text='こんにちは。今日はよくお越しくださいました。ゆっくりお話ししましょう。今日はどんなことでお悩みですか？'), Conversation(speaker='B', text='こんにちは。最近、漠然とした不安や焦りを感じていて。仕事もですが、日常生活全体がうまくいかない気がします。'), Conversation(speaker='A', text='そうなんですね。

In [68]:
# テキスト対話生成関数
def gen_txt_dialogue():
    resp = agent.invoke({"messages": [{"role": "user", "content": prompt_txt}]})
    return resp

In [69]:
resp = gen_txt_dialogue()

In [89]:
print(type(resp["messages"][-1].content))
print(resp["structured_response"].conversation)

<class 'str'>
[Conversation(speaker='A', text='こんにちは、本日はどうぞよろしくお願いいたします。今日はどんなことでも、お話したいことを聞かせてくださいね。'), Conversation(speaker='B', text='はい、よろしくお願いいたします。最近、仕事のことでも、なんだかこう、漠然とした不安というか、焦りのようなものを感じていて…。'), Conversation(speaker='A', text='そうなんですね。漠然とした不安や焦り、ですか。ええ、なるほど。もう少し、その不安や焦りについて、どんな時に感じられるのか、お話しいただけますか？'), Conversation(speaker='B', text='はい。特に何か具体的な問題があるわけじゃないんですけど、このままでいいのかな、とか、もっと何かできることがあるんじゃないか、とふと思ってしまって。'), Conversation(speaker='A', text='なるほど、このままでいいのかな、というお気持ちなんですね。はい。それはお仕事中だけでなく、日常生活の中でも感じられることはありますか？'), Conversation(speaker='B', text='ええ、そうですね。休日も、何かしていないと落ち着かないというか…。結局何もできなかった、って自分を責めてしまうことも多くて。'), Conversation(speaker='A', text='そうなんですね。休日も落ち着かず、ご自身を責めてしまうこともあるんですね。それはとてもお辛いですね。'), Conversation(speaker='B', text='はい。何をしていても、心から楽しめているのか、って考えてしまうんです。趣味も前ほど集中できなくなってしまって。'), Conversation(speaker='A', text='ええ、なるほど。心から楽しめていないと感じてしまう。以前はそうではなかった、ということですね。何かきっかけなど、思い当たることはありますか？'), Conversation(speaker='B', text='うーん、きっかけははっきりしないんですが、たぶん、今の仕事に就いてから、ずっとこんな調子かもしれません。'), Conversa

In [66]:
from typing import Any
from langchain_core.documents import Document
from langchain.agents.middleware import AgentMiddleware, AgentState


class State(AgentState):
    context: list[Document]


class RetrieveDocumentsMiddleware(AgentMiddleware[State]):
    state_schema = State

    def before_model(self, state: AgentState) -> dict[str, Any] | None:
        last_message = state["messages"][-1]
        retrieved_docs = vector_store.similarity_search(last_message.text)

        docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

        augmented_message_content = (
            f"{last_message.text}\n\n"
            "Use the following context to answer the query:\n"
            f"{docs_content}"
        )
        return {
            "messages": [last_message.model_copy(update={"content": augmented_message_content})],
            "context": retrieved_docs,
        }


debug_agent = create_agent(
    model,
    tools=[],
    middleware=[RetrieveDocumentsMiddleware()],
)

In [67]:
for step in debug_agent.stream(
    {"messages": [{"role": "user", "content": prompt_txt}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

メンタルヘルスケアカウンセリングのセッションをシミュレーションしてください。
シミュレーションしたい「段階」と「テーマ」:
【段階：初期】信頼関係を築きつつ、悩みの背景を深掘りするシーン

役割定義:
A (カウンセラー): メンタルヘルスケアの専門知識を持つ経験豊富なカウンセラー。傾聴と共感の姿勢を基本とし、クライエントの言葉を促すように、優しく、自然な話し言葉（「〜ですね」「〜でしたか」など）を使います。
B (クライエント): 仕事上の悩みだけでなく、日常生活全般に対して漠然とした不安や焦りを感じている人物。

対話の要件:
スタイル: 実際の会話の文字起こしのように、堅苦しくない自然な「話し言葉」を使用してください。
相槌 (あいづち): カウンセラー（A）は、クライエント（B）の話を促し、共感を示すため、「ええ」「はい」「そうなんですね」「なるほど」といった細かな相槌を頻繁に、適切なタイミングで挿入してください。
構成: 会話が途中で途切れるのではなく、初回のヒアリングとして「一区切り」がつき、自然に終了する流れにしてください（例：次回の約束、今回のまとめなど）。
分量: 会話の往復は合計12〜20ターン程度、全体の文字数が合計500〜800文字程度になるように構成してください。

================================ Human Message =================================

メンタルヘルスケアカウンセリングのセッションをシミュレーションしてください。
シミュレーションしたい「段階」と「テーマ」:
【段階：初期】信頼関係を築きつつ、悩みの背景を深掘りするシーン

役割定義:
A (カウンセラー): メンタルヘルスケアの専門知識を持つ経験豊富なカウンセラー。傾聴と共感の姿勢を基本とし、クライエントの言葉を促すように、優しく、自然な話し言葉（「〜ですね」「〜でしたか」など）を使います。
B (クライエント): 仕事上の悩みだけでなく、日常生活全般に対して漠然とした不安や焦りを感じている人物。

対話の要件:
スタイル: 実際の会話の